In [109]:
import numpy as np
import pandas as pd
import cv2
import win32com.client
import os
from ppe_detect import YOLOv8_ObjectDetector
import sys
from email_utils import send_email_with_attachment

In [110]:
ppe_labels = ["Arc Flash Suit", "Helmet", "Safety shoes", "Electric hand gloves", "Voltage regulator"]
ppe_detector = YOLOv8_ObjectDetector(model_file=r"D:/runs/detect/train15/weights/best.pt", labels=ppe_labels)

print(ppe_detector)


In [111]:
results_df = pd.DataFrame(columns=['frame_index', 'class', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max'])

In [112]:
video_path = r'D:/Saikrishna/Downloads/yolo_videos/ppe_undress.mp4'
cap = cv2.VideoCapture(video_path)

In [113]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('D:/runs/detect/train15/weights/best.pt')

# Define path to video file
source = 'D:/Saikrishna/Downloads/yolo_videos'

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects

print(results)

<generator object BasePredictor.stream_inference at 0x0000012E9B21B640>


In [114]:
if not cap.isOpened():
    print("Error opening video capture.")
    sys.exit()

In [115]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
out2 = cv2.VideoWriter('output_video2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [116]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
while True:
    ret, frame = cap.read()
    if not ret:
        break 
    detections = ppe_detector.predict_img(frame)
    display_img = ppe_detector.default_display(show_conf=True, pil=False, example_frame=frame) 
    
    # Display the frame with detections
    cv2.imshow('Frame with Detections', display_img)
    # Write the frame with detections to the output video
    out.write(display_img)
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the video capture and writer objects
cap.release()
out.release()
# Close all OpenCV windows
cv2.destroyAllWindows()